In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from tqdm import tqdm
from datetime import datetime

from labdata.schema import *

matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
#matplotlib.rcParams.update({'font.size': 18})

In [ ]:
Procedure() & 'procedure_type = "chronic implant"'

In [ ]:
#TODO: need to add neural data for MM010 and several JC mice so they will appear on the plot
Procedure() & 'procedure_type = "chronic explant"'

In [ ]:
#TODO: MUST ADD JC141 subject and insertions and explant
#Subject().insert1(dict(subject_name='JC141',
#                       #subject_dob=, # TODO: add DOB
#                       subject_gender='M',
#                       strain_name='R26-LNL-GtACR1-Fred-Kv2.1',
#                       user_name='couto'))
#
#to_insert = dict(subject_name='JC140',
#                 procedure_type='chronic implant',
#                 procedure_datetime='2023-12-05 18:00:00',
#                 user_name='couto',)
#Procedure().insert1(to_insert, skip_duplicates=True) 
#
#to_insert = dict(subject_name='JC140',
#                 procedure_type='chronic explant',
#                 procedure_datetime='2023-12-20 13:00:00',
#                 user_name='couto',)
#Procedure().insert1(to_insert, skip_duplicates=True) 

In [ ]:
EphysRecording.ProbeSetting()  & 'subject_name = "MM010"' # FIXME: missing MM010 right now.

In [ ]:
# Get all the implantation and explantation data from the database
procedures = EphysRecording.ProbeSetting() * Procedure() & 'procedure_type LIKE "chronic%"'
procedures = pd.DataFrame(procedures.fetch('probe_id', 'subject_name', 'procedure_datetime','procedure_type', as_dict=True)).drop_duplicates()
#print(f'There are {(len(procedures.probe_id.unique()))} probes in the database')
procedures.head()

In [ ]:
# Plot a Gantt chart for all probes used
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import  matplotlib.dates as mdates

color = list(cm.rainbow(np.linspace(0, 1, len(procedures.probe_id.unique()))))
color = list(cm.tab20(np.linspace(0, 1, len(procedures.probe_id.unique()))))
prbs, starts, ends, heights, cols = [], [], [], [], []
for prb_ind,prb in enumerate(procedures.probe_id.unique()):
    implant_times = procedures[(procedures.probe_id == prb) & (procedures.procedure_type == 'chronic implant')].procedure_datetime.values
    explant_times = procedures[(procedures.probe_id == prb) & (procedures.procedure_type == 'chronic explant')].procedure_datetime.values
    for implant in implant_times:
        explant = explant_times[explant_times > implant]
        explant = explant[0] if len(explant) else np.datetime64(datetime.now())
        starts.append(implant)
        #widths.append(np.datetime64(explant) - implant)
        ends.append(explant)
        heights.append(prb_ind)
        #cols.append(mpl_colors[prb_ind % len(mpl_colors)])
        cols.append(color[prb_ind])
    prbs.append(prb)

widths = np.array(ends) - np.array(starts)

fig, ax = plt.subplots(figsize=(10,5))
plt.barh(heights, widths, left=starts, color=cols)
plt.ylabel('Probe number')
plt.xlabel('Date')
plt.yticks(range(len(prbs)), prbs) # apply serial numbers
plt.gca().xaxis.set_major_locator(mdates.MonthLocator())
plt.grid(which='major', axis='x', linestyle='--')
plt.xticks(rotation=-45)
plt.show()

In [ ]:
implants = EphysRecording.ProbeSetting() * Procedure() & 'procedure_type LIKE "chronic_implant"'
mice = implants.fetch('subject_name', order_by='procedure_datetime') # sort by implant date
mice = pd.unique(mice)[::-1] #pd.unique preserves order unlike np.unique
mice

In [ ]:
# similar to plot above, but put each mouse on a separate row
procedures = EphysRecording.ProbeSetting() * Procedure() & 'procedure_type LIKE "chronic%"'
probe_ids = np.unique(procedures.fetch('probe_id'))
color = list(cm.tab20(np.linspace(0, 1, len(probe_ids))))
label_locations, prbs, starts, ends, heights, cols = [], [], [], [], [], []

c = 0
for m in mice:
    query = procedures & f'subject_name = "{m}"'
    prbs = np.unique(query.fetch('probe_id'))
    label_locations.append(c + len(prbs) // 2)
    for p in prbs:
        implant_date = (query & f'probe_id = "{p}"' & 'procedure_type = "chronic implant"').fetch('procedure_datetime')[0]
        explant_dates = (query & f'probe_id = "{p}"' & 'procedure_type = "chronic explant"').fetch('procedure_datetime')
        explant_date = explant_dates[0] if len(explant_dates) else datetime.now()
        starts.append(implant_date)
        ends.append(explant_date)
        heights.append(c)
        cols.append(color[np.where(probe_ids == p)[0][0]])
        c += 1 
    c +=.75

widths = np.array(ends) - np.array(starts)

fig, ax = plt.subplots(figsize=(10,5))
plt.barh(heights, widths, left=starts, height=1, color=cols)
plt.legend()
plt.ylabel('Animal ID')
plt.xlabel('Date')
plt.yticks(label_locations, mice) # apply serial numbers
plt.gca().xaxis.set_major_locator(mdates.MonthLocator())
plt.grid(which='major', axis='x', linestyle='--')
plt.xticks(rotation=-45)
plt.show()

In [ ]:
#TODO: add extractions to db and plot probe extraction success 
ProbeExtraction()